In [1]:
import pandas as pd
import numpy as np
import pulp
import os
os.getcwd()
os.chdir("..")

# Process data

In [2]:
raw_workmode = pd.read_excel('InputData.xlsx', sheet_name="Mode")
raw_demand = pd.read_excel('InputData.xlsx', sheet_name="Demand")
workmode = {(row['Mode'].item(), row['Shift'].item()): row['Capa'].item() for _, row in raw_workmode.iterrows()}
demand = {(row['Week'].item(), row['Wday'].item()): row['Demand'].item() for _, row in raw_demand.iterrows()}

In [3]:
WeekIndex = [1,2,3,4]
WeekDayIndex = [1,2,3,4,5,6,7]
ShiftIndex = [1,2,3]

In [4]:
numS = 22
numC = 12
FC = {
    (i, j): raw_workmode.loc[(raw_workmode['Mode'] == i) & (raw_workmode['Shift'] == j), 'Hours'].item() * 8.3
    for i in [1, 2]
    for j in ShiftIndex
    if not raw_workmode.loc[(raw_workmode['Mode'] == i) & (raw_workmode['Shift'] == j), 'Hours'].empty}

# Algorithm

In [33]:
# Create the model
model = pulp.LpProblem("3PL_Workforce_Allocation", pulp.LpMinimize)

# Define binary decision variables x (3D) and y (2D)
m_conso = pulp.LpVariable.dicts("m_conso", (ShiftIndex, WeekIndex, WeekDayIndex), cat='Binary')
m_sepa = pulp.LpVariable.dicts("m_sepa", (ShiftIndex, WeekIndex, WeekDayIndex), cat='Binary')

# Objective function: 
model += pulp.lpSum(numC*FC[1, sh] * m_conso[sh][w][d] for sh in ShiftIndex for w in WeekIndex for d in WeekDayIndex) + \
         pulp.lpSum(numS*FC[2, sh] * m_sepa[sh][w][d] for sh in ShiftIndex for w in WeekIndex for d in WeekDayIndex)

# Constraint
# Can't work 3 consecutive days
for w in WeekIndex:
    for d in range(1, len(WeekDayIndex) - 1):
        model += pulp.lpSum(m_conso[3][w][d+t] for t in [0,1,2]) <= 2
for w in WeekIndex:
    for d in range(1, len(WeekDayIndex) - 1):
        model += pulp.lpSum(m_sepa[3][w][d+t] for t in [0,1,2]) <= 2

# Can't use the same mode within a week
for w in WeekIndex:
    model += pulp.lpSum(m_sepa[sh][w][d] for sh in ShiftIndex for d in WeekDayIndex) <= 7
for w in WeekIndex:
    model += pulp.lpSum(m_conso[sh][w][d] for sh in ShiftIndex for d in WeekDayIndex) <= 7
for w in WeekIndex:
    model += pulp.lpSum(m_sepa[sh][w][d] for sh in ShiftIndex for d in WeekDayIndex) + pulp.lpSum(m_conso[sh][w][d] for sh in ShiftIndex for d in WeekDayIndex) >= 7

# Satisfy demand
for w in WeekIndex:
    for d in WeekDayIndex:
        for sh in ShiftIndex:
            model += m_conso[sh][w][d]*workmode[1, sh] + m_sepa[sh][w][d]*workmode[2, sh] >= demand[w, d]*(m_sepa[sh][w][d] + m_conso[sh][w][d])

# One shift only
for w in WeekIndex:
    for d in WeekDayIndex:
#         model += pulp.lpSum(m_sepa[sh][w][d] for sh in ShiftIndex) <= 1
#         model += pulp.lpSum(m_conso[sh][w][d] for sh in ShiftIndex) <= 1
        model += pulp.lpSum(m_conso[sh][w][d] for sh in ShiftIndex) + pulp.lpSum(m_sepa[sh][w][d] for sh in ShiftIndex) == 1
        
        

# Solve the problem using COIN_CMD without the tol argument
solver = pulp.PULP_CBC_CMD(gapRel=0.00001)  # Built-in solver
model.solve(solver)

# Collect the results into a list of dictionaries for x_ijk (3D)
results_conso = []
for sh in ShiftIndex:
    for w in WeekIndex:
        for d in WeekDayIndex:
            results_conso.append({
                'shift': sh,
                'week': w,
                'wday': d,
                'choose': m_conso[sh][w][d].varValue
            })

# Collect the results into a list of dictionaries for y_ij (2D)
results_sepa = []
for sh in ShiftIndex:
    for w in WeekIndex:
        for d in WeekDayIndex:
            results_sepa.append({
                'shift': sh,
                'week': w,
                'wday': d,
                'choose': m_sepa[sh][w][d].varValue
            })

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/lamjackie/miniconda3/envs/ych-wf-allocation/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/4v/qgjl0ptd02j55mwtql93llqr0000gn/T/a7add80ce37f4670af5f282f79f8d5ee-pulp.mps -ratio 1e-05 -timeMode elapsed -branch -printingOptions all -solution /var/folders/4v/qgjl0ptd02j55mwtql93llqr0000gn/T/a7add80ce37f4670af5f282f79f8d5ee-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 169 COLUMNS
At line 1466 RHS
At line 1631 BOUNDS
At line 1800 ENDATA
Problem MODEL has 164 rows, 168 columns and 792 elements
Coin0008I MODEL read with 0 errors
ratioGap was changed from 0 to 1e-05
Option for timeMode changed from cpu to elapsed
Continuous objective value is 31261.8 - 0.00 seconds
Cgl0003I 28 fixed, 0 tightened bounds, 7 strengthened rows, 7 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 0 strengthened rows, 7 substitutions
Cgl0004I processed model

In [34]:
df_results_conso=pd.DataFrame(results_conso)
df_results_conso['mode'] = "c"
df_results_sepa = pd.DataFrame(results_sepa)
df_results_sepa['mode'] = "s"
result = pd.concat([df_results_sepa, df_results_conso])
result_pivot=pd.DataFrame(result.pivot_table(index=["mode", "shift"], columns=["week", "wday"], values=["choose"], aggfunc="sum").reset_index(drop=False))
result_pivot

mode shift choose                                     ...                 \
week                 1                                  2  ...    3             
wday                 1    2    3    4    5    6    7    1  ...    5    6    7   
0       c     1    0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  ...  0.0  0.0  0.0   
1       c     2    1.0  1.0  1.0  1.0  1.0  1.0  1.0  0.0  ...  0.0  0.0  0.0   
2       c     3    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
3       s     1    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
4       s     2    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  1.0  1.0  1.0   
5       s     3    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   

                                         
week    4                                
wday    1    2    3    4    5    6    7  
0     0.0  0.0  0.0  0.0  0.0  0.0  0.0  
1     0.0  0.0  0.0  0.0  0.0  0.0  0.0  
2     0.0  0.0  0.0  0.0  0.0  0.0  0.0  
3     0.0  0.0  0.0  0.0  0.0  0.0  0.0  
4     1.0  1.0  1.0  1.0  1.0  1.0  1.0  
5     0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[6 rows x 30 columns]

In [24]:
raw_workmode

,Mode,Shift,Capa,Hours
0,1,1,7182,8
1,1,2,9453,10
2,1,3,11261,12
3,2,1,11271,8
4,2,2,13923,10
5,2,3,16576,12


In [25]:
raw_demand


,Week,Wday,Demand
0,1,1,7857
1,1,2,7857
2,1,3,7857
3,1,4,7857
4,1,5,7857
5,1,6,7857
6,1,7,7857
7,2,1,5000
8,2,2,5000
9,2,3,5000


In [35]:
with pd.ExcelWriter("result.xlsx") as writer:
   
    # use to_excel function and specify the sheet_name and index 
    # to store the dataframe in specified sheet
    result_pivot.to_excel(writer, sheet_name="Pivot")
    result.to_excel(writer, sheet_name="Results", index=False)
    raw_workmode.to_excel(writer, sheet_name="Workforce", index=False)
    raw_demand.to_excel(writer, sheet_name="Demand", index=False)